[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nils-holmberg/socs-qmd/blob/main/jnb/lecture5_cv2.ipynb)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# image dataset

In [ ]:
!gdown https://drive.google.com/uc?id=1sBsckDIyQJ-zTsEpjCntYMIJcbBpF3wk

In [ ]:
!unzip content-images.zip

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# For demonstration, creating a sample grayscale image
#image = np.random.randint(0, 256, (200, 200), dtype=np.uint8)
# Read the image using OpenCV
image_path = 'images/ibs-92.jpg'  # Replace with your image path
image = cv2.imread(image_path)

# Initialize ORB detector
orb = cv2.ORB_create()

# Detect ORB keypoints and descriptors in the grayscale version of the image
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
keypoints, descriptors = orb.detectAndCompute(gray_image, None)

# Draw keypoints on the original color image
orb_image = cv2.drawKeypoints(image, keypoints, None, color=(0, 255, 0), flags=0)

# Display the color image with ORB features
plt.figure(figsize=(6, 6))
plt.imshow(cv2.cvtColor(orb_image, cv2.COLOR_BGR2RGB))
plt.title('ORB Features on Color Image')
plt.axis('off')
plt.show()


# color clustering

In [ ]:
from sklearn.cluster import KMeans
from collections import Counter
from matplotlib import pyplot as plt
import cv2

In [ ]:
# Utility function, rgb to hex
def rgb2hex(rgb):
    hex = "#{:02x}{:02x}{:02x}".format(int(rgb[0]), int(rgb[1]), int(rgb[2]))
    return hex
print(rgb2hex([255, 0, 0]))

In [ ]:
def image_color_cluster(path, k=6):
    # load image
    img_bgr = cv2.imread(path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # resize image to speed up processing time
    resized_img_rgb = cv2.resize(img_rgb, (64, 64), interpolation=cv2.INTER_AREA)
    resized_img_rgb = img_rgb

    # reshape the image to be a list of pixels
    img_list = resized_img_rgb.reshape((resized_img_rgb.shape[0] * resized_img_rgb.shape[1], 3))

    # cluster the pixels and assign labels
    clt = KMeans(n_clusters=k)
    labels = clt.fit_predict(img_list)

    # count labels to find most popular
    label_counts = Counter(labels)
    total_count = sum(label_counts.values())

    # subset out most popular centroid
    center_colors = list(clt.cluster_centers_)
    ordered_colors = [center_colors[i]/255 for i in label_counts.keys()]
    color_labels = [rgb2hex(ordered_colors[i]*255) for i in label_counts.keys()]

    #print(label_counts.values())
    #print(color_labels)

    # plots
    plt.figure(figsize=(14, 8))
    #plt.subplot(221)
    #plt.imshow(img_rgb)
    #plt.axis('off')

    #plt.subplot(222)
    plt.pie(label_counts.values(), labels=color_labels, colors=ordered_colors, startangle=90)
    plt.axis('equal')
    plt.show()

In [ ]:
image_path = 'images/ibs-92.jpg'  # Replace with your image path
image_color_cluster(image_path, k=5)

# video dataset

In [ ]:
!gdown https://drive.google.com/uc?id=1T9S32UwmDnUd6YbxRYfyG7-gud8jh2Y_


In [ ]:
import cv2
import os

# Path to the video file
video_path = 'content-video.mp4'

# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Directory to save frames
save_dir = 'images-video'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

frame_idx = 0
while True:
    ret, frame = cap.read()

    # Break the loop if there are no more frames
    if not ret:
        break

    # Save every 100th frame
    if frame_idx % 100 == 0:
        # Filename with 5 leading zeroes
        save_path = os.path.join(save_dir, f'frame_{frame_idx:05d}.jpg')
        cv2.imwrite(save_path, frame)
        print(f'Saved frame {frame_idx:05d}')

    frame_idx += 1

# Release the VideoCapture object
cap.release()


# image ocr

In [ ]:
!pip install -q easyocr


In [ ]:
import easyocr

In [ ]:
#reader = easyocr.Reader(['ch_tra', 'en', 'sv'])
reader = easyocr.Reader(['en'])

In [ ]:
result = reader.readtext('images-video/frame_00400.jpg')
result

In [ ]:
reader.readtext('images-video/frame_00400.jpg', detail=0)

# image normalization

In [ ]:
import cv2
import os
import numpy as np

def resize_and_crop(img, size):
    # Resize image to maintain aspect ratio
    h, w, _ = img.shape
    if h > w:
        new_h, new_w = size * h / w, size
    else:
        new_h, new_w = size, size * w / h

    new_h, new_w = int(new_h), int(new_w)
    resized_img = cv2.resize(img, (new_w, new_h))

    # Crop the center of the image
    startx = new_w//2 - size//2
    starty = new_h//2 - size//2
    return resized_img[starty:starty+size, startx:startx+size]

def normalize_image(img):
    # Normalize pixel values to [0, 1]
    return img / 255.0

directory = 'images'
output_directory = 'images-normalize'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(directory):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
#    if filename.endswith('.jpg'):
        img = cv2.imread(os.path.join(directory, filename))
        img = resize_and_crop(img, 256)
        normalized_img = normalize_image(img)

        # Convert the normalized image back to 8-bit format
        img_to_save = (normalized_img * 255).astype(np.uint8)

        # Save the normalized image
        output_path = os.path.join(output_directory, filename)
        cv2.imwrite(output_path, img_to_save)


In [ ]:
import os
import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def extract_orb_features(image):
    """Extract ORB features from an image."""
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    if descriptors is None:
        return np.zeros((1, orb.descriptorSize()), dtype=np.float32)
    return descriptors

def calculate_similarity(descriptor_list):
    """Calculate similarity scores using feature matching."""
    num_images = len(descriptor_list)
    similarity_matrix = np.zeros((num_images, num_images))
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    for i in range(num_images):
        for j in range(i, num_images):
            if i == j:
                similarity_matrix[i, j] = 1
            else:
                # Ensure the descriptors are in the correct type
                desc1 = descriptor_list[i].astype(np.uint8)
                desc2 = descriptor_list[j].astype(np.uint8)

                matches = bf.match(desc1, desc2)
                score = len(matches) / max(len(desc1), len(desc2))
                similarity_matrix[i, j] = score
                similarity_matrix[j, i] = score

    return similarity_matrix

# Directory containing images
image_directory = 'images-normalize'  # Replace with your directory path

# Load and process images
image_files = [f for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
orb_descriptors = []

for file in image_files:
    image_path = os.path.join(image_directory, file)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    descriptors = extract_orb_features(image)
    orb_descriptors.append(descriptors)

# Calculate the similarity matrix
similarity_matrix = calculate_similarity(orb_descriptors)

# Plotting the similarity matrix
plt.figure(figsize=(10, 8))
sns.heatmap(similarity_matrix, annot=True, cmap='coolwarm')
plt.title('ORB Feature Similarity Matrix')
plt.xlabel('Image Index')
plt.ylabel('Image Index')
plt.show()


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def compute_histogram(image, bins=256):
    """Compute the color histogram for an image."""
    histogram = [cv2.calcHist([image], [i], None, [bins], [0, 256]) for i in range(3)]
    return np.concatenate(histogram).flatten()

def calculate_similarity(hist_list):
    """Calculate histogram similarity matrix."""
    num_images = len(hist_list)
    similarity_matrix = np.zeros((num_images, num_images))

    for i in range(num_images):
        for j in range(num_images):
            similarity = cv2.compareHist(hist_list[i], hist_list[j], cv2.HISTCMP_CORREL)
            similarity_matrix[i, j] = similarity

    return similarity_matrix

# Directory containing images
image_directory = 'images'  # Replace with your directory path
image_directory = 'images-normalize'  # Replace with your directory path

# Load and process images
image_files = [f for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
histograms = []

for file in image_files:
    image_path = os.path.join(image_directory, file)
    image = cv2.imread(image_path)
    if image is not None:
        hist = compute_histogram(image)
        histograms.append(hist)

# Calculate the similarity matrix
similarity_matrix = calculate_similarity(histograms)

# Plotting the similarity matrix
plt.figure(figsize=(10, 8))
sns.heatmap(similarity_matrix, annot=True, cmap='coolwarm')
plt.title('Image Histogram Similarity Matrix')
plt.xlabel('Image Index')
plt.ylabel('Image Index')
plt.show()


In [ ]:
import os
import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

# Function to compute the color histogram and similarity matrix
# ... (same functions `compute_histogram` and `calculate_similarity` as before)

# Directory containing images
image_directory = 'images'  # Replace with your directory path
image_directory = 'images-normalize'  # Replace with your directory path

# Load and process images
image_files = [f for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
histograms = []

for file in image_files:
    image_path = os.path.join(image_directory, file)
    image = cv2.imread(image_path)
    if image is not None:
        hist = compute_histogram(image)
        histograms.append(hist)

# Calculate the similarity matrix
similarity_matrix = calculate_similarity(histograms)

# Perform hierarchical clustering
Z = linkage(1 - similarity_matrix, method='average')

# Plot dendrogram
plt.figure(figsize=(12, 8))
dendrogram(Z, labels=image_files, orientation='right')
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Distance')
plt.ylabel('Image')
plt.show()


# image object detection

In [ ]:
!pip install -q transformers
!pip install -q timm


In [ ]:
from transformers import pipeline
model = pipeline("object-detection")

In [ ]:
import glob

rows_list = []
for img in sorted(glob.glob('images-video/frame_*.jpg')):
  dict1 = {}
  res = model(img)
  dict1.update({"image": img, "result": res})
  rows_list.append(dict1)

df = pd.DataFrame(rows_list)
df.head()

In [ ]:
rows_list = []
for index, row in df.iterrows():
  for i in row.result:
    dict1 = {}
    dict1.update({"image":row.image})
    dict1.update(i)
    rows_list.append(dict1)

df_res = pd.DataFrame(rows_list)
df_res.head()

In [ ]:
df_res.label.value_counts().plot(kind='bar')

In [ ]:
df_res[df_res.image=="images-video/frame_00100.jpg"]

In [ ]:
import cv2

img = cv2.imread('images-video/frame_00100.jpg')
for index,row in df_res[df_res.image=="images-video/frame_00100.jpg"].iterrows():
  x1,y1,x2,y2 = list(row.box.values())
  cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

from google.colab.patches import cv2_imshow
cv2_imshow(img)
#cv2.imshow("display", img)
#cv2.imwrite("objects.png", img)

# image content inference

In [ ]:
!pip install -q mediapipe

In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, enable_segmentation=True)
mp_drawing = mp.solutions.drawing_utils

# Read an image
image_path = 'images-video/frame_02800.jpg'  # Replace with the path to your image
image_path = 'images-video/frame_00400.jpg'  # Replace with the path to your image
image = cv2.imread(image_path)

# Check if image is loaded
if image is None:
    print("Error: Image not found.")
else:
    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and find the pose
    results = pose.process(image_rgb)

    # Draw pose landmarks on the image
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Convert back to BGR for displaying with OpenCV
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Display the image
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Release resources
pose.close()
